In [1]:
import pandas as pd
import spacy
from functools import reduce

In [2]:
def read_chunk(fname, chunk=100, rows=1000):
    df = pd.read_csv(fname, header=None, skiprows=1, chunksize=chunk, nrows=rows)
    return df

In [3]:
def extract_orgs(doc):
    for entity in doc.ents:
        if entity.label_ == "ORG":
            orgs.append(entity.text)

In [4]:
def process(chunk):
    for df in chunk:
        text = reduce(lambda x, y: x + " " + y, df[10])
        doc = nlp(text)
        extract_orgs(doc)

In [5]:
orgs = []

In [6]:
nlp = spacy.load("en_core_web_sm")

In [7]:
chunk = read_chunk("../data/amazon_reviews_sample.csv")